## Оценка работы модели RuBioRoBERTa в исходном виде

In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("alexyalunin/RuBioRoBERTa")
model = AutoModelForMaskedLM.from_pretrained("alexyalunin/RuBioRoBERTa")

Попробуем субъективно оценить качество работы модели. Для этого возьмем несколько предложений из медицинских протоколов (и просто из головы). В комментариях ожидаемые ответы

In [37]:
sentences = [
    "У пациента наблюдается повышенная", # температура
    "Кожные покровы физиологической окраски, умеренной", # влажности
    "Язык влажный, у корня обложен серым", # налетом
    "Жалобы на периодические головные", # боли
    "В области нижних конечностей имеется расширенная сосудистая", # сеть
    "Шейка матки расположена по проводной оси", # таза
    "Молекулярная диагностика наследственных", # заболеваний
    "Инфекционные заболевания: ветряная оспа, паротит, кандидоз, кандидозный", # баланит
    "Общемозговая симптоматика уменьшилась на фоне", # лечения
    "Умеренно выраженная возрастная дизритмия при", # гипервентиляции
    "Предварительный диагноз: Синдром вегетососудистой", # дистонии
    "На фоне признаков церебральной ангиодистонии отмечается умеренное увеличение скорости", # кровотока
    "Показания к пренатальному кариотипированию", # плода
    "Сохраняется умеренно выраженный вертеброгенный болевой", # синдром
    "КДГ с исследованием сосудов экстракраниального" # бассейна
]

Для каждого предложения выведем три наиболее подходящих варианта

In [39]:
for sent in sentences:
    text_with_mask = sent + " " + tokenizer.mask_token + "."
    input_ids = tokenizer.encode(text_with_mask, return_tensors="pt")
    with torch.no_grad():
        logits = model(input_ids).logits
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    masked_token_logits = logits[0, mask_token_index, :]
    tokens = torch.topk(masked_token_logits, 3, dim=1).indices[0].tolist()
    print(sent + " (" + ", ".join([tokenizer.decode([x]) for x in tokens])  + " )")

У пациента наблюдается повышенная ( температура,  чувствительность,  лихорадка )
Кожные покровы физиологической окраски, умеренной ( степени,  тяжести,  плотности )
Язык влажный, у корня обложен серым ( цветом,  слоем,  песком )
Жалобы на периодические головные ( боли,  боль,  приступы )
В области нижних конечностей имеется расширенная сосудистая ( сеть,  система,  зона )
Шейка матки расположена по проводной оси ( справа,  слева,  сердца )
Молекулярная диагностика наследственных ( болезней,  заболеваний,  нарушений )
Инфекционные заболевания: ветряная оспа, паротит, кандидоз, кандидозный ( синдром,  шок,  рак )
Общемозговая симптоматика уменьшилась на фоне ( лечения,  терапии,  препарата )
Умеренно выраженная возрастная дизритмия при ( БА,  СД,  РА )
Предварительный диагноз: Синдром вегетососудистой ( системы,  реакции,  болезни )
На фоне признаков церебральной ангиодистонии отмечается умеренное увеличение скорости ( потока,  ходьбы,  роста )
Показания к пренатальному кариотипированию 

Полученные результаты, хоть и не полностью соответствуют ожидаемым, демонстрируют, что модель в целом "понимает" медицинскую тематику предложений и предлагает +- адекватные варианты продолжения.